sudo /opt/conda/envs/interns_env/bin/python -m pip install transformers torch

In [2]:
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import torch
import numpy as np

/opt/conda/envs/interns_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Скачиваем модель и токенизатор
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = AutoModel.from_pretrained('distilbert-base-uncased')

def embed_text(text):
    # Токенизируем
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    # Получаем результат модели
    outputs = model(**inputs)
    # Возвращаем последнее скрытое состояние результата модели
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

# Данные
query = "What is the capital of France?"
documents = ["Tokyo is the capital of Japan.", "Paris is the capital of France.", "The capital of Germany is Berlin."]

# Получаем эмбединг запроса
query_embedding = embed_text(query)
# Получаем эмбединг каждого документа, транспонируя матрицу
document_embeddings = [embed_text(doc).T for doc in documents]

# Вычисляем косинусное сходство через матричное умножение
similarities = (query_embedding @ document_embeddings).squeeze()

print("Query:", query)
for i, doc in enumerate(documents):
    print(f"Document {i+1}: {doc} (Similarity: {round(similarities[i], 2)})")

Query: What is the capital of France?
Document 1: Tokyo is the capital of Japan. (Similarity: 57.130001068115234)
Document 2: Paris is the capital of France. (Similarity: 65.79000091552734)
Document 3: The capital of Germany is Berlin. (Similarity: 61.68000030517578)


1.
```python
return outputs.last_hidden_state.mean(dim=1).detach().numpy()
    
    Размерность outputs.last_hidden_state -> (batch_size, sequence_length, hidden_size), где "sequence_length" - кол-во слов\токенов.

    
    BERT-tokenizer токенизирует по словам, поэтому чтобы получить векторное представление (эмбединг) всего предложения, то мы можем взять средний эмбединг всех слов - .mean(dim=1) - это и будет эмбединг всего предложения

```

2.
```python
    Транспонирование приводит к тому, что document_embedding.𝑇 изменяет размерность с (1, D) на (D, 1),
    что позволяет выполнить матричное умножение: query_embedding (1, D) на document_embedding.𝑇 (D, 1), 
    результатом чего будет скалярное произведение (1, 1).
    
    @ - матричное умножение
    .squeeze() - удаляет измерения -> превращает [[значение1], [значение2]] в [значение1, значение2]
```

In [4]:
# Скачиваем модель и токенизатор
tokenizer = AutoTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-ru')
model = AutoModelForSeq2SeqLM.from_pretrained('Helsinki-NLP/opus-mt-en-ru')

def translate(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    # Модель принимает в себя токены и возвращает токены
    outputs = model.generate(**inputs)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Пример инференса
text = "Hello, how are you?"
translation = translate(text)
print("Original:", text)
print("Translation:", translation)

/opt/conda/envs/interns_env/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Original: Hello, how are you?
Translation: Привет, как дела?


In [5]:
inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
print(f'Входные токены: {inputs["input_ids"]}')
outputs = model.generate(**inputs)

print("\nДекодирование выходных токенов:")
for output in outputs[0]:
    print(f'Token: {output:<6}  Word: {tokenizer.decode(output)}')

Входные токены: tensor([[ 160, 5270,    2,  508,   55,   33,   19,    0]])

Декодирование выходных токенов:
Token: 62517   Word: <pad>
Token: 543     Word: При
Token: 1877    Word: вет
Token: 2       Word: ,
Token: 69      Word: как
Token: 1082    Word: дела
Token: 19      Word: ?
Token: 0       Word: </s>


In [6]:
# Скачиваем модель и создаем pipeline для анализа тональности
sentiment_analyzer = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

# Примеры текста для анализа тональности
texts = [
    "I love this movie!",
    "This is the worst product I have ever bought.",
    "The food was delicious and the service was excellent.",
    "I am not happy with the way things turned out.",
    "What an amazing experience!",
    "I feel very disappointed with the performance.",
    "The weather is terrible today.",
    "I don't like the new design at all.",
    "This is the best book I have read this year."
]

# Анализ тональности для каждого текста
for text in texts:
    result = sentiment_analyzer(text)[0]
    print(f"Text: {text}\nSentiment: {result['label']}\n")


Text: I love this movie!
Sentiment: POSITIVE

Text: This is the worst product I have ever bought.
Sentiment: NEGATIVE

Text: The food was delicious and the service was excellent.
Sentiment: POSITIVE

Text: I am not happy with the way things turned out.
Sentiment: NEGATIVE

Text: What an amazing experience!
Sentiment: POSITIVE

Text: I feel very disappointed with the performance.
Sentiment: NEGATIVE

Text: The weather is terrible today.
Sentiment: NEGATIVE

Text: I don't like the new design at all.
Sentiment: NEGATIVE

Text: This is the best book I have read this year.
Sentiment: POSITIVE



In [5]:
%%time
# Скачиваем модель и токенизатор
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained('microsoft/DialoGPT-small')

def chat_with_bot(prompt):
    inputs = tokenizer(prompt, return_tensors='pt', truncation=True, padding=True)
    outputs = model.generate(**inputs, max_length=50, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Пример инференса
prompt = "Does money buy happiness?"
response = chat_with_bot(prompt)
print("User:", prompt)
print("Bot:", response[len(prompt):])

User: Does money buy happiness?
Bot:  Buy happiness buy life.
CPU times: user 2.35 s, sys: 371 ms, total: 2.72 s
Wall time: 1 s
